In [1]:
from utils import utils_hackathon as uh
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import importlib

In [2]:
df_drias = pd.read_csv("data/drias_montpellier_RCP8.5_df.csv")
df_drias["T_Q"] = df_drias["T_Q"] - 273.15
df_mf = pd.read_csv("data/mf_montpellier.csv")
df_drias.head(5)

,DATE,T_Q
0,1951-01-01,4.10530
1,1951-01-02,1.32964
2,1951-01-03,2.46975
3,1951-01-04,1.80877
4,1951-01-05,1.30822


In [3]:
df_mf.head(5)

,Unnamed: 0,DATE,T_Q
0,0,1960-01-01,10.8
1,1,1960-01-02,9.8
2,2,1960-01-03,12.1
3,3,1960-01-04,10.7
4,4,1960-01-05,8.3


In [4]:
# ne pas inclure dans streamlit
seuil = 25
signe = "+"
periode_start = "07-01"
periode_end = "10-30"
indicateur = "Nb_jours_max"
dict_indicateurs = {
    "T_MAX": "Temperature maximale",
    "nb_episodes": "Nombre d'épisodes",
    "Nb_jours_max": "Nombre de jours où la température est > à 25 °C ",
}

In [5]:
importlib.reload(uh)
fig, df_mf_jour_conse, df_drias_jour_conse = uh.main_indic_nb_jour_consecutif(
    df_mf,
    df_drias,
    seuil,
    periode_start,
    periode_end,
    dict_indicateurs,
    signe,
)

In [6]:
df_mf

,Unnamed: 0,DATE,T_Q,Mois-Jour
0,0,1960-01-01,10.8,01-01
1,1,1960-01-02,9.8,01-02
2,2,1960-01-03,12.1,01-03
3,3,1960-01-04,10.7,01-04
4,4,1960-01-05,8.3,01-05
...,...,...,...,...
23949,23949,2024-02-25,8.0,02-25
23950,23950,2024-02-26,9.5,02-26
23951,23951,2024-02-27,10.3,02-27
23952,23952,2024-02-28,11.5,02-28


In [7]:
# ne pas inclure dans streamlit
signe = "+"
periode_start = "07-01"
periode_end = "10-30"
indicateur = "T_MOYENNE"
dict_indicateurs = {
    "T_MAX": "Temperature maximale",
    "T_MIN": "Température minimale",
    "T_MOYENNE": "Température moyenne",
    "nb_episodes": "Nombre d'épisodes",
    "Nb_jours_max": "Nombre de jours où la température est > à 25 °C ",
}

In [8]:
importlib.reload(uh)
fig, _, _ = uh.main_indic_temperature(
    df_mf=df_mf,
    df_drias=df_drias,
    indicateur=indicateur,
    periode_start="07-01",
    periode_end="10-30",
    dict_indicateurs=dict_indicateurs,
)
fig.show()

In [9]:
ind_dict = {
    "Température Max": "T_MAX",
    "Température Min": "T_MIN",
    "Température Moyenne": "T_MOYENNE",
}
dict_indicateurs["Nb_jours_max"] = (
    f"Nombre de jours où la température est > à {seuil} °C ",
)
ind = "Nb_jours_max"

In [10]:
metrique2000 = uh.prepa_df_metrique(df_drias_jour_conse, 2000, ind)
metrique2020 = uh.prepa_df_metrique(df_drias_jour_conse, 2030, ind)
metrique2050 = uh.prepa_df_metrique(df_drias_jour_conse, 2050, ind)
metrique2000, metrique2020, metrique2050

ValueError: cannot convert float NaN to integer

# Graphe série 

In [11]:
# hors streamlit
df_metier = pd.read_csv("data/qualite_vin.csv")
df_metier.head(1)

,var_buis,Année
0,3.0,1986


In [12]:
df_m = df_mf_jour_conse.rename(columns={ind: "index"})
df_d = df_drias_jour_conse.rename(columns={ind: "index"})
df_mf_jour_conse.head(1)

,Année,Nb_jours_max,ANOM_Nb_jours_max,rolling_avg,rolling_std,avg + std,avg - std
0,1951,33,11.7,NaN,NaN,NaN,NaN


In [13]:
df_m = df_m[["Année", "index"]]
df_d = df_d[["Année", "index"]]

In [14]:
df_m.head(1)

,Année,index
0,1951,33


In [15]:
df_metier.head(1)

,var_buis,Année
0,3.0,1986


In [16]:
df_indicateur = df_m.merge(df_metier, on="Année")[df_m.columns]

In [21]:
fig = go.Figure()
# Ajout de la courbe pour df_m
fig.add_trace(
    go.Scatter(
        x=df_indicateur["Année"],
        y=df_indicateur["index"],
        mode="lines+markers",
        name="Evolution de la température maximale (°C)",
    )
)

# Ajout de la courbe pour df_metier
fig.add_trace(
    go.Scatter(
        x=df_metier["Année"],
        y=df_metier["var_buis"],
        mode="lines+markers",
        name="Qualité du vin",
    )
)
fig.update_layout(
    legend=dict(
        title="Légende",
        # x=0.01, y=0.99,
        bgcolor="rgba(255, 255, 255, 0.5)",
        bordercolor="black",
        borderwidth=1,
    )
)
fig.update_layout(xaxis_title="Année", yaxis_title="Valeur")
fig.show()

In [22]:
import plotly.graph_objects as go

fig = go.Figure()

# Ajout de la courbe pour df_indicateur avec l'axe y principal
fig.add_trace(
    go.Scatter(
        x=df_indicateur["Année"],
        y=df_indicateur["index"],
        mode="lines+markers",
        name="Evolution de la température maximale (°C)",
        yaxis="y1",  # Utilise l'axe y principal
    )
)

# Ajout de la courbe pour df_metier avec l'axe y secondaire
fig.add_trace(
    go.Scatter(
        x=df_metier["Année"],
        y=df_metier["var_buis"],
        mode="lines+markers",
        name="Qualité du vin",
        yaxis="y2",  # Utilise l'axe y secondaire
    )
)

# Mise à jour de la mise en page pour ajouter un axe y secondaire
fig.update_layout(
    xaxis_title="Année",
    yaxis_title="Température maximale (°C)",
    yaxis=dict(
        title="Température maximale (°C)",  # Titre de l'axe y principal
        titlefont=dict(color="#1f77b4"),
        tickfont=dict(color="#1f77b4")
    ),
    yaxis2=dict(
        title="Qualité du vin",
        titlefont=dict(color="#ff7f0e"),
        tickfont=dict(color="#ff7f0e"),
        overlaying="y",  # Indique que cet axe est superposé à l'axe y principal
        side="right",  # Position de l'axe sur le côté droit
        range=[0, 5]  # Plage spécifique pour l'échelle de la qualité du vin
    ),
    legend=dict(
        title="Légende",
        bgcolor="rgba(255, 255, 255, 0.5)",
        bordercolor="black",
        borderwidth=1
    )
)

fig.show()


In [45]:
import plotly.graph_objects as go

fig = go.Figure()

# Ajout de la courbe pour df_m
fig.add_trace(
    go.Scatter(
        x=df_indicateur["Année"],
        y=df_indicateur["index"],
        mode="lines+markers",
        name="Température maximale annuelle",
        yaxis="y"
    )
)

# Ajout de la courbe pour df_metier
fig.add_trace(
    go.Scatter(
        x=df_metier["Année"],
        y=df_metier["var_buis"],
        mode="lines",
        name="Qualité du vin",
        yaxis="y2"
    )
)

## Personnalisation des légendes
fig.update_layout(
    title = {"text": "Evolution de la température maximale et de la qualité du vin en fonction des années"},
    legend=dict(
        title="Légende",
        #x=0.01, y=0.99,
        bgcolor="rgba(255, 255, 255, 0.5)",
        bordercolor="black",
        borderwidth=1
    ),
    legend2=dict(
        title="Légende 2",
        #x=0.01, y=0.95,
        bgcolor="rgba(255, 255, 255, 0.5)",
        bordercolor="black",
        borderwidth=1
    )
)

# Ajout du double axe y
fig.update_layout(
    yaxis2=dict(
        title="Qualité du vin",
        overlaying="y",
        side="right"
    )
)

## Ajout des labels pour les axes
fig.update_layout(
    xaxis_title="Année",
    yaxis_title="Indicateur Métier",
    yaxis2_title="Métier"
)


return fig
fig.show()

In [37]:
df_merge = df_indicateur.merge(df_metier, on = "Année")
res = df_merge.corr()
correlation = res.loc["var_buis", "index"]
correlation

,Année,index,var_buis
0,1986,46,3.0


0.27356287983620337

0.27356287983620337